# Basic Test Case for LGMC (Dynamic)

## Species (14)
EC0, EC-, LiEC+, LiEC0, LiEC_RO0, LiEC_RO-,
LiCO3-, Li2CO30, LEDC0, LEDC-, LEDC_minus_Li-, LEDC_plus_Li+, LEDC_plus_Li0
C2H40


## Overview:
The molecular thermodynamics are in "test_species_thermo.json". Non-electrochemical reactions are in "test_energy_barriers.json"; inner reorganization energies for reduction reactions are in "test_lambda_inner.json". For outer reorganization energies, we can just use a constant value for now 0.32 eV should be reasonable. For the hopping reactions (in "test_hopping_reactions.json"), I defined the barrier based on what Li was hopping from. For LiEC+, for instance, the barrier is 0.27 eV (based on the residence time of Li+ with EC solvation shells), but for LEDC0 it's 0.64 eV (based on values taken from the literature).

There are a couple of considerations not included in these files:

1. Phase: I didn't mark what phase each reaction can occur in. I think it's probably fine to allow all reactions to occur in either phase, though certain reactions will be unlikely to happen in one phase or another because of adsorption/desorption.
2. Adsorption/desorption: I didn't include adsorption or desorption reactions. In terms of species that we want to be able to adsorp, the most important are the solid products (LiCO3-, Li2CO30, LEDC0, LEDC-, LEDC_minus_Li-, LEDC_plus_Li+, LEDC_plus_Li0). These should adsorb very favorably (adsorption should be fast; desorption should be slow). Technically any of the EC-like species (EC0, EC-, LiEC+, LIEC0, LiEC_RO0, LiEC_RO-) should also be able to adsorb, but to keep the model simple we could just not allow that. In terms of desorption, the only thing that should desorb fast is C2H4. I don't have a good idea for rates for adsorption/desorption reactions; you might need to play around with it.
3. Lattice diffusion: Through the hopping reactions, we can capture lattice diffusion of Li+ (which is the most important). I didn't look up diffusion of CO3^2- or EDC^2- anions. To keep this model as simple as possible, we could just not include any lattice diffusion. Or if you can find some reasonable barrier/rate from the literature, that works too. I'm easy here.
4. Sizes: 
    1: C2H40, LiCO3-, Li2CO30
    2: EC0, EC-, LiEC+, LiEC0, LiEC_RO0, LiEC_RO-
    3: LEDC0, LEDC-, LEDC_minus_Li-, LEDC_plus_Li+, LEDC_plus_Li0

# Reactions Format

- Adsorption/Desorption/Diffusion
    - 2D array of species and associated rates [species, rate]
- Homogeneous solution chemical reactions
    - .json : {'reactants': ['EC', 'Li_plus'], 'products': ['LiEC_plus'], 'barrier': 0.0} 
- Homogeneous lattice chemical reactions
    - .json : {'reactants': ['EC', 'Li_plus'], 'products': ['LiEC_plus'], 'barrier': 0.0} 
    - includes recoordination reactions
- Oxidation/reduction 
??

In [1]:
import sqlite3
from itertools import product, combinations
import copy
import os
import pandas as pd

import numpy as np
from scipy.constants import pi, epsilon_0, elementary_charge

from monty.serialization import loadfn

from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN, metal_edge_extender

from atomate.qchem.database import QChemCalcDb

/Users/laurazichi/opt/anaconda3/lib/python3.9/site-packages/maggma/utils.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def make_df(path, table):
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT * FROM " + table, con)
    con.close()
    
    return df 

In [3]:
# ------------------- User defined -------------------- #
base_dir = "/Users/laurazichi/git/SQL/LGMC/SEI/simple_test"
kmc_path = "/Users/laurazichi/git/SQL/LGMC/SEI/simple_test"
# ----------------------------------------------------- #

# Room temperature (25 C) in Kelvin
ROOM_TEMP = 298.15

# Boltzmann constant in eV / K
KB = 8.617333262 * 10 ** -5

# Planck constant in eV * s
PLANCK = 4.135667696 * 10 ** -15

# Simulation variables
dielectric = 18.5
refractive = 1.415
radius = 5.0
electrode_distance = 0.0
adiabatic=True
decay_constant=1.2

# kMC parameters
factor_zero = 1.0
factor_two = 1.0
factor_duplicate = 0.5

# Miscellaneous constants
# For now, don't apply a solvent correction
# TODO: Do we do this?
# solv_corr_factor = 0.68

# Ideas to avoid hopping:
# Have all reduction reactions create new Li+ (charge neutrality argument)
#   Should we also be including a constant influx of EC?
#   Kind of nice to have EC be finite and Li+ be infinite - could potentially lead to "completion"?
# Have all coordination reactions create new Li+
#   Seems less physical


# For now, try with hopping and without
# # Residence time of Li+ with EC, from Tingzheng, is roughly 5 ns
# min_hopping_barrier = 0.266
# # Assume nothing takes longer than 1 us to recoordinate (arbitrary; need to experiment)
# max_hopping_barrier = 0.402

In [4]:
# Functions
def eyring(dg_barrier, kappa=1.0, temperature=ROOM_TEMP):
    if dg_barrier <= 0:
        return kappa * KB * temperature / PLANCK
    else:
        return kappa * KB * temperature / PLANCK * np.exp(-dg_barrier / (KB * temperature))


def lambda_outer(r=radius, d=electrode_distance, eps=dielectric, n=refractive):
    lambda_o = abs(elementary_charge) / (8 * pi * epsilon_0)
    lambda_o *= (1 / r - 1 / (2 * d)) * 10 ** 10
    lambda_o *= 1 / n ** 2 - 1 / eps
    return lambda_o


def barrier_redox(dg, lambda_i, dq=1, e_free=-1.4, r=radius, d=electrode_distance, eps=dielectric, n=refractive):
    lambda_o = lambda_outer(r=r, d=d, eps=eps, n=n)
    lambda_total = lambda_i + lambda_o

    delta_g = dg + dq * e_free

    dg_barrier = lambda_total / 4 * (1 + delta_g / lambda_total) ** 2

    return dg_barrier

In [5]:
def create_state_sqlite(path, stmts, species, insert_into_factors = "INSERT INTO factors VALUES (?,?,?)",
                        factor_zero = 1, factor_two = 1, factor_duplicate = 0.5):
    '''Create sqlite of initial species concentrations'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()

    rn_cur.execute(
        insert_into_factors,
        (factor_zero, factor_two, factor_duplicate)
    )

    for i in range(len(species)):
        rn_cur.execute(
            "INSERT INTO initial_state VALUES (?,?)",
            (i, initial_state.get(i,0)))

    rn_con.commit()
    rn_con.close()

In [6]:
def create_reaction_sqlite(path, stmts, num_species):
    ''' Create an empty sqlite, name, with a table for the metadata and 
    reactions'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()
    
    rn_cur.execute(
        insert_metadata,
        (num_species, 0))

    rn_con.commit()
    
    rn_con.close()

In [7]:
def add_homogeneous_reactions(path, chem_rxns, species_thermo, mapping, insert_reaction, phase):
    ''' Add homogeneous chemical lattice reactions to specified table'''
    
    # connect to table
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])

    reactions = set()
    print("ADDING CHEMICAL REACTIONS")

    for rxn in chem_rxns:
        rcts = rxn["reactants"]
        pros = rxn["products"]

        g_rct = sum([species_thermo[x]['g'] for x in rcts])
        g_pro = sum([species_thermo[x]['g'] for x in pros])
        dg_forward = round(g_pro - g_rct, 2)
        dg_reverse = -1 * dg_forward

        num_reactants = len(rcts)
        num_products = len(pros)

        indices_reactants = [mapping[x] for x in rcts]
        if num_reactants == 1:
            indices_reactants.append(-1)

        indices_products =  [mapping[x] for x in pros]
        if num_products == 1:
            indices_products.append(-1)

        barrier_forward = rxn["barrier"]
        barrier_reverse = barrier_forward - dg_forward

        k_forward = eyring(barrier_forward)
        k_reverse = eyring(barrier_reverse)

        if (indices_reactants[0], indices_reactants[1], indices_products[0], indices_products[1]) not in reactions:
            reactions.add((indices_reactants[0], indices_reactants[1], indices_products[0], indices_products[1]))

            reactant_two = indices_reactants[1]
            product_two = indices_products[1]
            num_react = num_reactants
            num_prod = num_products
            
            if(phase == 'L'):           
                if(num_reactants > num_products):
                    reactant_two = indices_reactants[1]
                    product_two = 0
                    num_prod = 2
                elif(num_reactants < num_products):
                    reactant_two = 0
                    product_two = indices_products[1]
                    num_react = 2
            
            rn_cur.execute(insert_reaction,
                       (reaction_index,
                        num_react,
                        num_prod,
                        indices_reactants[0],
                        reactant_two,
                        indices_products[0],
                        product_two,
                        phase,
                        phase,
                        phase,
                        phase,
                        dg_forward,
                        1,
                        k_forward,
                        0,
                        0,
                        0,
                        phase))
                

            reaction_index += 1
            rn_con.commit()

        if (indices_products[0], indices_products[1], indices_reactants[0], indices_reactants[1]) not in reactions:
            reactions.add((indices_products[0], indices_products[1], indices_reactants[0], indices_reactants[1]))

            
            reactant_two = indices_reactants[1]
            product_two = indices_products[1]
            num_react = num_reactants
            num_prod = num_products
            
            if(phase == 'L'):
                if(num_reactants > num_products):
                    reactant_two = indices_reactants[1]
                    product_two = 0
                    num_prod = 2
                elif(num_reactants < num_products):
                    reactant_two = 0
                    product_two = indices_products[1]
                    num_react = 2
            
            
            rn_cur.execute(insert_reaction,
                           (reaction_index,
                            num_prod,
                            num_react,
                            indices_products[0],
                            product_two,
                            indices_reactants[0],
                            reactant_two,
                            phase,
                            phase,
                            phase,
                            phase,
                            dg_reverse,
                            1,
                            k_reverse,
                            0,
                            0,
                            0,
                            phase))
            reaction_index += 1
            rn_con.commit()

    print("DONE ADDING CHEMICAL REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close()
    

In [8]:
def add_adsorption(path, adsorp_rxns, mapping, insert_reaction):
    '''Add adsorption reaction to sqlite file. adsorp_rnxs 2D array
    of species and corresponding rate [species, rate]'''
    
    # connect to table
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])
    
    for rnx in adsorp_rxns:
        
        rn_cur.execute(insert_reaction,
                           (reaction_index,
                            2,
                            1,
                            0,
                            mapping[rnx[0]],
                            mapping[rnx[0]],
                            -1,
                            "L",
                            "S",
                            "L",
                            "N",
                            -100,
                            1,
                            rnx[1],
                            0,
                            0,
                            0,
                            "A"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING ADSORPTION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))
    
    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close()    

In [9]:
def add_desorption(path, desorp_rxns, mapping, insert_reaction):
    '''Add adsorption reaction to sqlite file. adsorp_rnxs 2D array
    of species and corresponding rate [species, rate]'''
    
    # connect to table
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])
    
    for rnx in desorp_rxns:
        
        rn_cur.execute(insert_reaction,
                           (reaction_index,
                            1,
                            2,
                            mapping[rnx[0]],
                            -1,
                            0,
                            mapping[rnx[0]],
                            "L",
                            "N",
                            "L",
                            "S",
                            -100,
                            1,
                            rnx[1],
                            0,
                            0,
                            0,
                            "D"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING DESORPTION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))
    
    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close() 

In [10]:
def add_diffusion(path, diff_rxns, mapping, insert_reaction):
    ''' Add diffusion reactions to sqlite file (name). diff_rxns 2D array of 
        species, rate'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])

    for rnx in diff_rxns:

        rn_cur.execute(insert_reaction,
                       (reaction_index,
                        2,
                        2,
                        mapping[rnx[0]],
                        0,
                        0,
                        mapping[rnx[0]],
                        "L",
                        "L",
                        "L",
                        "L",
                        0,
                        1,
                        rnx[1],
                        0,
                        0,
                        0,
                        "F"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING DIFFUSION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close() 
    

In [11]:
def add_reduction(path, rxns, mapping, insert_reaction, prefactor):
    ''' Add reduction reactions to sqlite file (name). rxns '''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])

    for rnx in rxns:

        rn_cur.execute(insert_reaction,
                       (reaction_index,
                        1,
                        1,
                        mapping[rnx['reactants'][0]],
                        -1,
                        mapping[rnx['products'][0]],
                        -1,
                        "L",
                        "N",
                        "L",
                        "N",
                        rnx['dg'],
                        prefactor,
                        0,
                        1.2,
                        0.32 + rnx['lambda_inner'],
                        0.5,
                        "R"))
        reaction_index += 1
        rn_con.commit()
        
        rn_cur.execute(insert_reaction,
                       (reaction_index,
                        1,
                        1,
                        mapping[rnx['reactants'][0]],
                        -1,
                        mapping[rnx['products'][0]],
                        -1,
                        "S",
                        "N",
                        "S",
                        "N",
                        rnx['dg'],
                        prefactor,
                        0,
                        1.2,
                        0.32 + rnx['lambda_inner'],
                        0.5,
                        "R"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING REDUCTION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close() 
    

In [12]:
create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# it is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.
create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id                     INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants             INTEGER NOT NULL,
            number_of_products              INTEGER NOT NULL,
            reactant_1                      INTEGER NOT NULL,
            reactant_2                      INTEGER NOT NULL,
            product_1                       INTEGER NOT NULL,
            product_2                       INTEGER NOT NULL,
            phase_reactant_1                CHAR(1) NOT NULL,
            phase_reactant_2                CHAR(1) NOT NULL,
            phase_product_1                 CHAR(1) NOT NULL,
            phase_product_2                 CHAR(1) NOT NULL,
            dG                              REAL NOT NULL,
            prefactor                       REAL NOT NULL,
            rate                            REAL NOT NULL,
            electron_tunneling_coefficient  REAL NOT NULL,
            reorganization_energy           REAL NOT NULL,
            charge_transfer_coefficient     REAL NOT NULL,
            type                            CHAR(1) NOT NULL
    );
"""

insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""
create_trajectories_table = """
    CREATE TABLE trajectories (
            seed                INTEGER NOT NULL,
            step                INTEGER NOT NULL,
            time                REAL NOT NULL,
            reaction_id         INTEGER NOT NULL,
            site_1_mapping      INTEGER NOT NULL,
            site_2_mapping      INTEGER NOT NULL
    );
"""

create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""

create_interrupt_state = """
    CREATE TABLE interrupt_state (
            seed                    INTEGER NOT NULL,
            species_id              INTEGER NOT NULL,
            quantity                INTEGER NOT NULL,
            site_mapping            INTEGER NOT NULL,
            edge                    INTEGER NOT NULL
            
    );
"""

create_interrupt_cutoff = """
    CREATE TABLE interrupt_cutoff (
            seed                    INTEGER NOT NULL,
            step                    INTEGER NOT NULL,
            time                    INTEGER NOT NULL, 
            maxk                   INTEGER NOT NULL
            
    );
"""

In [13]:
species_thermo = loadfn(os.path.join(base_dir, "test_species_thermo.json"))
chem_rxns = loadfn(os.path.join(base_dir, "test_energy_barriers.json"))
echem_rxns = loadfn(os.path.join(base_dir, "test_lambda_inner.json"))
hopping_reactions = loadfn(os.path.join(base_dir, "test_hopping_reactions.json"))

mapping = {m: i for i, m in enumerate(species_thermo, 1)}
chem_rxns[2]['products'] = ['C2H40', 'LiCO3-']

In [15]:
mapping

{'EC0': 1,
 'EC-': 2,
 'LiEC+': 3,
 'LiEC0': 4,
 'LiEC_RO0': 5,
 'LiEC_RO-': 6,
 'LiCO3-': 7,
 'Li2CO30': 8,
 'LEDC0': 9,
 'LEDC-': 10,
 'LEDC_minus_Li-': 11,
 'C2H40': 12,
 'LEDC_plus_Li+': 13,
 'LEDC_plus_Li0': 14}

In [17]:
species_thermo

{'EC0': {'e': -9318.79298119549,
  'h': -9316.565887747689,
  's': 0.0031102900725,
  'g': -9317.493220732806,
  'composition': 'C3 H4 O3'},
 'EC-': {'e': -9319.531457513942,
  'h': -9317.403928039741,
  's': 0.0032055610001999997,
  'g': -9318.35966605195,
  'composition': 'C3 H4 O3'},
 'LiEC+': {'e': -9520.805446396735,
  'h': -9518.484556400635,
  's': 0.0035374698216,
  'g': -9519.539253027946,
  'composition': 'C3 H4 Li1 O3'},
 'LiEC0': {'e': -9522.939756872549,
  'h': -9520.69440713865,
  's': 0.0034026941988000003,
  'g': -9521.708920414021,
  'composition': 'C3 H4 Li1 O3'},
 'LiEC_RO0': {'e': -9523.966981998674,
  'h': -9521.806495808474,
  's': 0.0037227212568,
  'g': -9522.916425151188,
  'composition': 'C3 H4 Li1 O3'},
 'LiEC_RO-': {'e': -9526.87086244333,
  'h': -9524.723689031829,
  's': 0.0036540758865000003,
  'g': -9525.81315175739,
  'composition': 'C3 H4 Li1 O3'},
 'LiCO3-': {'e': -7389.35190917139,
  'h': -7388.7484109916895,
  's': 0.0029208756837,
  'g': -7389.6192

In [17]:
# Create sqlite file with initial state
initial_state = {mapping["LiEC+"]: 10000}

stmts = [create_initial_state_table, create_trajectories_table, 
         create_factors_table, create_interrupt_cutoff, 
         create_interrupt_state]

# Create initial state database 
create_state_sqlite("state.sqlite", stmts, species_thermo)

In [23]:
# Create reaction database and add reactions
stmts = [create_metadata_table, create_reactions_table]
create_reaction_sqlite('network.sqlite', stmts, len(species_thermo)+1)

In [24]:
# Add homogeneous reactions to reaction.sqlite
# All reactions can occur in either phase
add_homogeneous_reactions('network.sqlite', chem_rxns, species_thermo, mapping, insert_reaction, 'S')
add_homogeneous_reactions('network.sqlite', chem_rxns, species_thermo, mapping, insert_reaction, 'L')
add_homogeneous_reactions('network.sqlite', hopping_reactions, species_thermo, mapping, insert_reaction, 'L')
add_homogeneous_reactions('network.sqlite', hopping_reactions, species_thermo, mapping, insert_reaction, 'S')

ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 12 TOTAL REACTIONS
ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 24 TOTAL REACTIONS
ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 54 TOTAL REACTIONS
ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 84 TOTAL REACTIONS


In [25]:
# Add adsorption reactions
# Species which can adsorb: LiCO3-, Li2CO30, LEDC0, LEDC-, LEDC_minus_Li-, LEDC_plus_Li+, LEDC_plus_Li0
x = 1e4
rxns = [["LiCO3-", x], ["Li2CO30", x], ["LEDC0", x], ["LEDC-", x], ["LEDC_minus_Li-", x],
           ["LEDC_plus_Li+", x], ["LEDC_plus_Li0", x]]
add_adsorption('network.sqlite', rxns, mapping, insert_reaction)

DONE ADDING ADSORPTION REACTIONS; 91 TOTAL REACTIONS


In [26]:
rxns = [["LiCO3-", 1e1], ["C2H40", x]]
add_desorption('network.sqlite', rxns, mapping, insert_reaction)

DONE ADDING DESORPTION REACTIONS; 93 TOTAL REACTIONS


In [27]:
prefactor = 1e6
add_reduction('network.sqlite', echem_rxns, mapping, insert_reaction, prefactor)

DONE ADDING REDUCTION REACTIONS; 99 TOTAL REACTIONS
